In [30]:
import ast
import pandas as pd

def recommend_doctors(user_age: int, user_test: dict) -> set:
    """
    Returns set of recommended doctors based on age, test, and department.
    Department is automatically mapped based on test severity:
        - If any test result is 'severe' → Psychiatry
        - Else → Psychology
    """
    age_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\age_group_segment.csv")
    test_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\test_segment.csv")
    dept_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\dept_segment.csv")

    # -----------------------
    # Department Mapping (NEW)
    # -----------------------
    if any(result.lower() == "severe" for result in user_test.values()):
        user_department = "Psychiatry"
    else:
        user_department = "Psychology"

    # -----------------------
    # 1. Age → doctor list
    # -----------------------
    def get_age_segment(age: int) -> str:
        if 2 <= age <= 4:
            return "2-4"
        elif 5 <= age <= 12:
            return "5-12"
        elif 13 <= age <= 18:
            return "13-18"
        elif 19 <= age <= 30:
            return "19-30"
        elif 31 <= age <= 45:
            return "31-45"
        elif 46 <= age <= 60:
            return "46-60"
        else:
            return "61+"
    
    user_age_segment = get_age_segment(user_age)
    age_doctors = age_segment.loc[age_segment["age_group"] == user_age_segment, "doctor_id"].values[0]
    if isinstance(age_doctors, str):
        age_doctors = ast.literal_eval(age_doctors)
    age_doctors = set(age_doctors)

    # -----------------------
    # 2. Test → doctor list
    # -----------------------
    test_segment["test_combination"] = test_segment["test_combination"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    test_segment["doctors"] = test_segment["doctors"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    def get_doctors_for_test(user_test_dict, test_segment):
        matching_doctors = []
        for user_test in user_test_dict.keys():
            for _, row in test_segment.iterrows():
                if user_test in row["test_combination"]:
                    matching_doctors.extend(row["doctors"])
        return list(set(matching_doctors))  # remove duplicates

    test_doctors = set(get_doctors_for_test(user_test, test_segment))

    # -----------------------
    # 3. Department → doctor list
    # -----------------------
    dept_segment["doctors"] = dept_segment["doctors"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    dept_doctors = dept_segment.loc[
        dept_segment["department"].str.lower() == user_department.lower(), "doctors"
    ].values[0]
    dept_doctors = set(dept_doctors)

    # -----------------------
    # Final: Intersection
    # -----------------------
    common_doctors = age_doctors & test_doctors & dept_doctors
    return common_doctors


In [37]:
import pandas as pd

# -----------------------------
# Function to check availability (only for given common_doctors with load > 0)
# -----------------------------
def get_doctor_availability(user_date, common_doctors: set):
    """
    user_date       : str in 'YYYY-MM-DD' format
    common_doctors  : set of doctor IDs to check availability for

    Returns: dict {doctor_id: load} for doctors with load > 0 and not on leave
    """
    appointments_df = pd.read_csv(
        r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\doctor_appointment_load.csv"
    )
    leaves_df = pd.read_csv(
        r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Leavedata.csv"
    )
    leaves_df["Leave_Dates"] = pd.to_datetime(leaves_df["Leave_Dates"], format="%d-%m-%Y")

    # Convert input date
    user_date = pd.to_datetime(user_date, format="%Y-%m-%d")

    # Step 1: find doctors on leave that day
    doctors_on_leave = leaves_df.loc[
        leaves_df["Leave_Dates"] == user_date, "Doctor_ID"
    ].tolist()

    # Step 2: get column name that matches user_date in appointment load
    date_str = user_date.strftime("%Y-%m-%d (%A)")
    if date_str not in appointments_df.columns:
        raise ValueError(f"Date {date_str} not found in appointment load table")

    # Step 3: filter appointment load for that date
    result_df = appointments_df[
        ["Doctor ID", "Doctor Name", "Department", date_str]
    ].copy()
    result_df.rename(columns={date_str: "Load"}, inplace=True)

    # Step 4: filter only common_doctors
    result_df = result_df[result_df["Doctor ID"].isin(common_doctors)]

    # Step 5: remove doctors on leave
    result_df = result_df[~result_df["Doctor ID"].isin(doctors_on_leave)]

    # Step 6: convert "Leave" text to 0 load
    result_df["Load"] = pd.to_numeric(result_df["Load"], errors="coerce").fillna(0)

    # Step 7: keep only doctors with load > 0
    result_df = result_df[result_df["Load"] > 0]

    # Step 8: return dictionary
    return dict(zip(result_df["Doctor ID"], result_df["Load"]))


In [38]:
df = recommend_doctors(
    user_age=22,
    user_test={"AQ-10": "severe", "ASRS": "moderate"}
)
print(df)

{'D017', 'D018', 'D012', 'D014', 'D003', 'D013', 'D019', 'D001', 'D002', 'D004'}


In [39]:
availability = get_doctor_availability("2025-08-29", df)
print(availability)

{'D013': 33.0, 'D014': 47.0, 'D019': 61.0}


In [50]:
import pandas as pd

def compute_doctor_scores(doctor_availability: dict, 
                          patient_gender: str):
    """
    Compute doctor scores based on availability, load balancing, and gender affinity.

    Parameters:
        doctor_availability (dict): {doctor_id: appointments_today}
        patient_gender (str): "M" or "F"
        doctor_gender_map (dict): {doctor_id: "M"/"F"}

    Returns:
        pd.DataFrame: Doctor scores with columns [DoctorID, A, L, G, FinalScore]
        """
    max_daily_capacity = 60

    doctors_df = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\doctors.csv")  # or pd.read_excel("doctors.xlsx")
    doctor_gender_map = doctors_df.set_index("Doctor ID")["Gender"].to_dict()

    rows = []
    for doctor_id, load_today in doctor_availability.items():
        
        # Availability Score
        A = 1 if load_today < max_daily_capacity else 0
        
        # Load Balancing Score
        L = max(0, 1 - (load_today / max_daily_capacity))
        
        # Gender Affinity Score
        doctor_gender = doctor_gender_map.get(doctor_id, None)
        if doctor_gender is None:
            G = 0.5  # default if gender missing
        else:
            G = 1 if doctor_gender == patient_gender else 0.5
        
        # Final Score (weighted sum, you can adjust weights)
        FinalScore = (0.4 * A) + (0.4 * L) + (0.2 * G)
        
        rows.append({
            "Doctor ID": doctor_id,
            "AppointmentsToday": load_today,
            "Max load": A,
            "Load Balancing": L,
            "Gender affinity": G,
            "FinalScore": FinalScore
        })
    top_dr=pd.DataFrame(rows).sort_values("FinalScore", ascending=False).reset_index(drop=True)

    # Merge with doctor details
    top_dr_full = top_dr.merge(doctors_df, on="Doctor ID", how="left")

    return top_dr_full[["Name", "Gender", "Department","Available Days", "Available Slots", "FinalScore"]]

patient_gender = "Female"

scores_df = compute_doctor_scores(availability, patient_gender)
print(scores_df)

                  Name  Gender  Department Available Days  \
0  Dr. Ritika Malhotra  Female  Psychiatry    Fri,Sat,Tue   
1     Dr. Priya Mishra  Female  Psychiatry    Tue,Fri,Wed   
2   Dr. Shweta Ankathi  Female  Psychiatry    Thu,Mon,Fri   

                       Available Slots  FinalScore  
0  11:00-12:00,14:00-15:00,13:00-14:00    0.780000  
1  12:00-13:00,16:00-17:00,13:00-14:00    0.686667  
2  10:00-11:00,12:00-13:00,13:00-14:00    0.200000  


In [53]:
df=get_recommended_doctors(
    user_age=22,user_test={"AQ-10": "severe", "ASRS": "moderate"},user_date="2025-08-29",patient_gender = "Female")

print(df)

                  Name  Gender  Department Available Days  \
0  Dr. Ritika Malhotra  Female  Psychiatry    Fri,Sat,Tue   
1     Dr. Priya Mishra  Female  Psychiatry    Tue,Fri,Wed   
2   Dr. Shweta Ankathi  Female  Psychiatry    Thu,Mon,Fri   

                       Available Slots  FinalScore  
0  11:00-12:00,14:00-15:00,13:00-14:00    0.780000  
1  12:00-13:00,16:00-17:00,13:00-14:00    0.686667  
2  10:00-11:00,12:00-13:00,13:00-14:00    0.200000  


In [52]:
import ast
import pandas as pd

def get_recommended_doctors(user_age: int, user_test: dict, user_date: str, patient_gender: str):
    """
    Combines recommendation, availability, and scoring into one function.
    
    Parameters:
        user_age (int)       : Patient's age
        user_test (dict)     : Test results, e.g. {"AQ-10":"severe", "ASRS":"moderate"}
        user_date (str)      : Date in 'YYYY-MM-DD'
        patient_gender (str) : "M" or "F"

    Returns:
        pd.DataFrame with columns: [Name, Gender, Department, Available Days, Available Slots, FinalScore]
    """

    # -----------------------
    # Load all CSVs
    # -----------------------
    age_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\age_group_segment.csv")
    test_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\test_segment.csv")
    dept_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\dept_segment.csv")
    appointments_df = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\doctor_appointment_load.csv")
    leaves_df = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Leavedata.csv")
    doctors_df = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\doctors.csv")

    # -----------------------
    # Step 1: Department Mapping
    # -----------------------
    if any(result.lower() == "severe" for result in user_test.values()):
        user_department = "Psychiatry"
    else:
        user_department = "Psychology"

    # -----------------------
    # Step 2: Age Segment → doctor list
    # -----------------------
    def get_age_segment(age: int) -> str:
        if 2 <= age <= 4: return "2-4"
        elif 5 <= age <= 12: return "5-12"
        elif 13 <= age <= 18: return "13-18"
        elif 19 <= age <= 30: return "19-30"
        elif 31 <= age <= 45: return "31-45"
        elif 46 <= age <= 60: return "46-60"
        else: return "61+"
    
    user_age_segment = get_age_segment(user_age)
    age_doctors = age_segment.loc[age_segment["age_group"] == user_age_segment, "doctor_id"].values[0]
    if isinstance(age_doctors, str):
        age_doctors = ast.literal_eval(age_doctors)
    age_doctors = set(age_doctors)

    # -----------------------
    # Step 3: Test Segment → doctor list
    # -----------------------
    test_segment["test_combination"] = test_segment["test_combination"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    test_segment["doctors"] = test_segment["doctors"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    def get_doctors_for_test(user_test_dict, test_segment):
        matching_doctors = []
        for user_test in user_test_dict.keys():
            for _, row in test_segment.iterrows():
                if user_test in row["test_combination"]:
                    matching_doctors.extend(row["doctors"])
        return list(set(matching_doctors))

    test_doctors = set(get_doctors_for_test(user_test, test_segment))

    # -----------------------
    # Step 4: Department Segment → doctor list
    # -----------------------
    dept_segment["doctors"] = dept_segment["doctors"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    dept_doctors = dept_segment.loc[
        dept_segment["department"].str.lower() == user_department.lower(), "doctors"
    ].values[0]
    dept_doctors = set(dept_doctors)

    # -----------------------
    # Step 5: Common doctors
    # -----------------------
    common_doctors = age_doctors & test_doctors & dept_doctors
    if not common_doctors:
        return pd.DataFrame(columns=["Name", "Gender", "Department", "Available Days", "Available Slots", "FinalScore"])

    # -----------------------
    # Step 6: Availability
    # -----------------------
    leaves_df["Leave_Dates"] = pd.to_datetime(leaves_df["Leave_Dates"], format="%d-%m-%Y")
    user_date_dt = pd.to_datetime(user_date, format="%Y-%m-%d")

    doctors_on_leave = leaves_df.loc[leaves_df["Leave_Dates"] == user_date_dt, "Doctor_ID"].tolist()
    date_str = user_date_dt.strftime("%Y-%m-%d (%A)")
    if date_str not in appointments_df.columns:
        raise ValueError(f"Date {date_str} not found in appointment load table")

    result_df = appointments_df[["Doctor ID", "Doctor Name", "Department", date_str]].copy()
    result_df.rename(columns={date_str: "Load"}, inplace=True)
    result_df = result_df[result_df["Doctor ID"].isin(common_doctors)]
    result_df = result_df[~result_df["Doctor ID"].isin(doctors_on_leave)]
    result_df["Load"] = pd.to_numeric(result_df["Load"], errors="coerce").fillna(0)
    result_df = result_df[result_df["Load"] > 0]
    doctor_availability = dict(zip(result_df["Doctor ID"], result_df["Load"]))

    if not doctor_availability:
        return pd.DataFrame(columns=["Name", "Gender", "Department", "Available Days", "Available Slots", "FinalScore"])

    # -----------------------
    # Step 7: Compute Scores
    # -----------------------
    max_daily_capacity = 60
    doctor_gender_map = doctors_df.set_index("Doctor ID")["Gender"].to_dict()

    rows = []
    for doctor_id, load_today in doctor_availability.items():
        A = 1 if load_today < max_daily_capacity else 0
        L = max(0, 1 - (load_today / max_daily_capacity))
        doctor_gender = doctor_gender_map.get(doctor_id, None)
        G = 1 if doctor_gender == patient_gender else 0.5 if doctor_gender else 0.5
        FinalScore = (0.4 * A) + (0.4 * L) + (0.2 * G)

        rows.append({
            "Doctor ID": doctor_id,
            "AppointmentsToday": load_today,
            "FinalScore": FinalScore
        })

    top_dr = pd.DataFrame(rows).sort_values("FinalScore", ascending=False).reset_index(drop=True)
    top_dr_full = top_dr.merge(doctors_df, on="Doctor ID", how="left")

    return top_dr_full[["Name", "Gender", "Department", "Available Days", "Available Slots", "FinalScore"]]
